# Data Analysis Notebook
Importación de librerías


Lectura de ficheros de datos en formato csv

In [1]:
wind_ava=pd.read_csv('wind_ava.csv.gz', compression = "gzip")

wind_comp=pd.read_csv('wind_comp.csv.gz', compression = "gzip")

NameError: name 'pd' is not defined

Visualización de las primeras muestras de datos de los ficheros iniciales

In [ ]:
wind_ava.head()

In [ ]:
wind_ava['datetime'] = pd.to_datetime(wind_ava['datetime'])
wind_ava.head()

Si modificamos el índice para que este sea la variable *datetime* nos será más simple operar con ello.

In [ ]:
#wind_ava = wind_ava.set_index('datetime')
#wind_ava.head()


Actualmente, el fichero tiene 551 columnas. No las necesitamos todas: sólo usaremos la información sobre la localización de Sotavento, es decir, el número 13 de la cuadrícula de 5x5. 

A continuación, modificamos el fichero de datos para eliminar las columnas que no necesitamos y quedarnos únicamente con las columnas de variables relativas a la posición 13 de la cuadrícula. Para ello, filtramos las columnas según aquellas que contengan el número 13 en su nombre.
Además, nos interesa saber el valor de la energía en cada momento en esa localización, por lo que también filtraremos de forma que no se elimine la variable 'energy'.

In [ ]:
for c in wind_ava.columns:
    if not c.endswith('.13') and c != 'energy' and c != 'datetime':
        wind_ava = wind_ava.drop(c, axis = 1)

In [ ]:
import pandas as pd
import matplotlib
import sklearn
import time

In [ ]:
wind_ava.head()

Como se puede observar, hemos pasado de operar con 551 columnas, a tener sólo 23 columnas en nuestro dataset.
En último lugar, es necesario modificar la variable 'datetime' y convertir los datos de la primera columna a formato fecha (pd.to_datetime()) para facilitar su manejo y visualización. Podemos comprobar el resultado de esta modificación en los diagramas de la siguiente sección. 

## Descripción general de los datos

In [ ]:
wind_ava

In [ ]:
df_shape = wind_ava.shape
df_shape

Nuestro dataset tiene 23 características y 4748 instancias.

In [2]:
df_description = wind_ava.describe()
df_description

NameError: name 'wind_ava' is not defined

Con esta información resumida sobre los datos disponibles podemos averiguar la media, el mínimo y el máximo y la desviación típica de cada variable, entre otras cosas. 

En primer lugar, podemos observar que todas nuestras variables son numéricas. 

También observamos que ningún valor de desviación estándar o desviación típica (std) es igual a 0, por lo que ninguna de las variables de nos interesan (las del sector 13 del mapa) son constantes, y por lo tanto no es necesario modificar el dataset más allá de lo relaizado hasta ahora.



# Preprocesado de datos
En primer lugar, queremos identificar y eliminar los datos faltantes en nuestro dataset, es decir, aquellos que son *NA*.
Para ello identificamos el número de datos faltantes por cada característica.

In [3]:
wind_ava.isna().sum()

NameError: name 'wind_ava' is not defined

Como podemos ver, no hay ningún valor perdido en nuestros datos, por lo que no necesitamos eliminar ninguna característica. Lo habríamos hecho en el caso de que, para alguna característica, más del 80% de los datos fueran faltantes.

A continuación comprobamos que la desviación típica de todas las variables sea diferente de 0, lo que supondría que todas son significativas y aportan valor al modelo. En caso de que este dato tomara valor nulo supondría que esa variable es constante, y posiblemente se valoraría su eliminación del conjunto de datos.

In [4]:
wind_ava.std()

NameError: name 'wind_ava' is not defined

## Visualización de los datos
Una vez se ha preprocesado y limpiado el conjunto de los datos, se procede a la creación de diferentes diagramas para recolectar información sobre la distirbución de los datos.
Con un diagrama de barras se pueden apreciar las distribuciones de energía recogidas en cada instante de tiempo (representados como el índice del conjunto de datos).

In [5]:
wind_ava['energy'].plot(rot=20)

NameError: name 'wind_ava' is not defined

Para poder identificar información sobre los outliers utilizamos un diagrama de caja o boxplot.

In [6]:
wind_ava['energy'].plot.box()

NameError: name 'wind_ava' is not defined

Se puede observar que hay bastantes outliers de energía generada. A pesar de que la mediana se encuentra alrededor de 500, y el tercer cuartil alrededor de 1000, existen bastantes valores fuera del rango intercuartílico, y todos ellos superando el valor de energía generada de 2500, mientras que por debajo del primer cuartil prácticamente no se percibe la existencia de outliers.

También se puede observar la frecuencia de energía en los diferentes rangos temporales (calulados con el números de divisiones *bins*) mediante un histograma.

In [7]:
wind_ava['energy'].plot.hist(bins=25)

NameError: name 'wind_ava' is not defined

Se puede observar que la distribución se acumula a la izquierda de los valores temporales. Para solucionar un posible sesgo, se puede convertir la variable *energy* a logarítmica de forma que en el gráfico se aprecie como una distribución normal, en lugar de la actual que asemeja una distribución logarítmica.
Para realizar esta operación, utilizaremos la librería NumPy. Para ello inicialmente importamos la librería mencionada como se muestra a continuación:

In [8]:
import numpy as np

In [9]:
wind_ava['log_energy'] = np.log(wind_ava['energy'])

wind_ava['log_energy'].plot.hist(bins=40)

NameError: name 'wind_ava' is not defined

En este caso, el resultado no es especialmente relevante, ya que la operación convierte un sesgo a la izquierda en un sesgo a la derecha, de forma que el gráfico resultante no asemeja prácticamente una distirbución normal.

# Método de evaluación Outer & Inner
Tras haber estudiado y limpiado el conjunto de datos, se explica la estrategia que se llevará a cabo para evaluar el modelo (*outer*) y para definir el ajuste de hiperparámetros (*inner*).

*`Outer`* 
Para estimar el rendimiento esperado del modelo con datos futuros (para los que no ha sido entrenado), separamos una parte de los datos para evaluar (partición test) el modelo resultante del inner.

*`Inner`*
Emplearemos Grid Search con Cross Validation y Time Series Splits.

Para evlauar el rendimiento de los diferentes modelos se deben escoger una serie de métricas
RMSE y MAE y dummy regressor para escalar los datos y en base a ese dummy calcular RSE y RAE.
Para hacer el modelo dummy se le quita la media al real
ESto nos permite observar el error en términos absoluto y relativo
para cross validation MSE y de cara a la evaluación del modelo final tomaríamos todas esta medidas.

In [ ]:
wind_ava.head()

In [ ]:
from sklearn import preprocessing, neighbors, metrics, pipeline, linear_model, svm
from sklearn.pipeline import Pipeline
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor

In [ ]:
#División de conjuntos de evaluación outer. 
#Dividimos en train y test aprovechando una falta de datos para el año 2009.
train= wind_ava[wind_ava['datetime'].dt.year<2009]
X_train= train.drop(columns='energy')
y_train= train['energy']
print("train shape is:", train.shape,"\n X_train shape:",X_train.shape, "\n y_train shape: ", y_train.shape)

In [ ]:
test= wind_ava[wind_ava['datetime'].dt.year>2008]
X_test= test.drop(columns='energy')
y_test= test['energy']
print("test shape is:", test.shape,"\n X_test shape:",X_test.shape, "\n y_train shape: ", y_test.shape)

In [ ]:
print(train.shape, test.shape, wind_ava.shape)

In [ ]:
y_test.head()

In [ ]:
X_train = X_train.set_index('datetime')
X_train.head()

In [ ]:
X_test = X_test.set_index('datetime')

# Selección de Scaler
En esta sección se emplea KNN y CV para encontrar el método de escalado que minimice el error entre las tres opciones que ofrece sklearn: MinMax, Standarization y Robust. 
Para ello, empleamos un sencillo crossvalidation con un split adecuado para series temporales y medimos el error empleando la métrica escogida (MSE).

In [ ]:
#1) Preparación de datos -usaremos el subset train. 
scalerCV= TimeSeriesSplit(n_splits=5)
scaler_eval= {}

In [ ]:
#2)Evaluación de scalers
#StandardScaler
from sklearn.preprocessing import StandardScaler
pipeline_std = sklearn.pipeline.Pipeline([('scaler', StandardScaler()),
                                           ('KNN', neighbors.KNeighborsRegressor())])

score_std = cross_val_score(pipeline_std, X_train, y_train,cv= scalerCV, scoring='neg_mean_squared_error')

scaler_eval["StandardScaler"] = -score_std.mean()

In [ ]:
#MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
pipeline_mnmx = sklearn.pipeline.Pipeline([('scaler', MinMaxScaler()),
                                           ('KNN', neighbors.KNeighborsRegressor())])

score_mnmx = cross_val_score(pipeline_mnmx, X_train, y_train,cv= scalerCV, scoring='neg_mean_squared_error')

scaler_eval["MinMaxScaler"] = -score_mnmx.mean()

In [ ]:
#RobustScaler
from sklearn.preprocessing import RobustScaler
pipeline_rbst = sklearn.pipeline.Pipeline([('scaler', RobustScaler()),
                                           ('KNN', neighbors.KNeighborsRegressor())])

score_rbst = cross_val_score(pipeline_rbst, X_train, y_train,cv= scalerCV, scoring='neg_mean_squared_error')

scaler_eval["RobustScaler"] = -score_rbst.mean()

In [ ]:
print(scaler_eval)


Usando MSE o RMSE, el error más pequeño que hemos obtenido es el de RobustScaler con un valor de 137309.

In [10]:
pipeline_mnmx.fit(X_train, y_train)
y_pred = pipeline_mnmx.predict(X_test)
outer_score = metrics.mean_squared_error(y_test, y_pred)
print(outer_score)

NameError: name 'pipeline_mnmx' is not defined

# Selección de Método
En esta sección se miden los tiempos de entrenamiento de diferentes propuestas de modelo para solucionar el problema planeado. 
A continuación, se realiza una optimización e hiperparámetros (HPO) sobre dichos modelos candidatos y se realizan mediciones temporales del proceso. 
Finalmente, se emplean los datos recopilados (tiempo y scores de los modelos) para analizar y extraer conclusiones, escogiendo el modelo final que se empleará para solucionar el problema de predicción propuesto. 

# 1) mediciones iniciales
Se probarán y evaluarán modelos por defecto de KNN, árboles de regresión, regresión lineal y SMV.

In [11]:
time_split= TimeSeriesSplit(n_splits=5)

times= {}
#A. KNN

#knn train measurement
time0= time.time()
model= sklearn.pipeline.Pipeline([('scaler',RobustScaler()),
                                ('KNN', neighbors.KNeighborsRegressor())])
model.fit(X_train, y_train)

time1= time.time()
knn_train= time1 - time0

#knn cross validation measurement
time0= time.time()
knn_score= cross_val_score(model, X_train, y_train, cv= time_split, scoring='neg_mean_squared_error')
time1= time.time()
knn_cv= time1 - time0

times["KNN"]= {"train_time": knn_train, "cv_time":knn_cv, "total_time":knn_train + knn_cv, "score": -knn_score.mean()}

print("KNN: ", times["KNN"])

#B. Árboles de regresión

#tree train measurement
time0= time.time()
model= sklearn.pipeline.Pipeline([('scaler',RobustScaler()),
                                ('RegTree', DecisionTreeRegressor())])
model.fit(X_train, y_train)

time1= time.time()
tree_train= time1 - time0

#tree cross validation measurement
time0= time.time()
tree_score= cross_val_score(model, X_train, y_train, cv= time_split, scoring='neg_mean_squared_error')
time1= time.time()
tree_cv= time1 - time0

times["Tree"]= {"train_time": tree_train, "cv_time":tree_cv, "total_time":tree_train + tree_cv, 
                "score": -tree_score.mean()}
print("Tree: ", times["Tree"])

#C. Regresor lineal: standard y lasso
#C.1. Regular
time0= time.time()
model= sklearn.pipeline.Pipeline([('scaler',RobustScaler()),
                                ('Linear Reg', linear_model.LinearRegression())])
model.fit(X_train, y_train)
time1= time.time()
linear_train= time1 - time0

time0= time.time()
linear_score= cross_val_score(model, X_train, y_train, cv= time_split, scoring='neg_mean_squared_error')
time1= time.time()
linear_cv= time1 - time0

times["Linear"]= {"train_time": linear_train, "cv_time":linear_cv, "total_time":linear_train + linear_cv, 
                "score": -linear_score.mean()}
print("Linear Reg: ", times["Linear"])

#C.2. Lasso
time0= time.time()
model= sklearn.pipeline.Pipeline([('scaler',RobustScaler()),
                                ('Lasso Reg', linear_model.Lasso())])
model.fit(X_train, y_train)
time1= time.time()
lasso_train= time1 - time0

time0= time.time()
lasso_score= cross_val_score(model, X_train, y_train, cv= time_split, scoring='neg_mean_squared_error')
time1= time.time()
lasso_cv= time1 - time0

times["Lasso"]= {"train_time": lasso_train, "cv_time":lasso_cv, "total_time":lasso_train + lasso_cv, 
                "score": -lasso_score.mean()}
print("Lasso Reg: ", times["Lasso"])

#D. SMV
time0= time.time()
model= sklearn.pipeline.Pipeline([('scaler',RobustScaler()),
                                ('SMV', svm.SVR())])
model.fit(X_train, y_train)
time1= time.time()
smv_train= time1 - time0

time0= time.time()
smv_score= cross_val_score(model, X_train, y_train, cv= time_split, scoring='neg_mean_squared_error')
time1= time.time()
smv_cv= time1 - time0

times["SMV"]= {"train_time": smv_train, "cv_time": smv_cv, "total_time":smv_train + smv_cv, 
                "score": -smv_score.mean()}
print("SMV: ", times["SMV"])

NameError: name 'TimeSeriesSplit' is not defined